<img src="../Img/banner-fa-49-2.jpg">

### <center> Финансовый университет
## <center>  Современные технологии <br>прикладного программирования и обработки данных
## <center> Тема 6. Методы группировки данных

20 апреля 2021 года

Лекция<br>
Студенты: Поток У18-7у<br>
Преподаватель: Смирнов Михаил Викторович, доцент Департамента Анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации

# Группировка данных

Данные можно представить себе как последовательность каких-то единичных наблюдений. Например, в наборе данных *FinancialSample2.csv* единичное наблюдение — это сделка по продаже конкретного товара.

In [ ]:
import pandas as pd
fs=pd.read_csv("../Data/FinancialSample2.csv", sep=';')
fs.head()

У каждого наблюдения есть какие-то атрибуты. Они могут быть разных типов:

- категориальные — например, название фирмы-производителя товара (признак *Product*);
- численные — например, размер партии товара в штуках (признак *UnitsSold*).

Решение задачи группировки предполагает разделение данных по некоторому признаку (атрибуту), после чего к каждому элементу этих разделенных данных мы можем применить агрегирующую операцию. Это такая операция, которая позволяет вычислить какой-либо показатель.

Затем мы можем оценить, как отличаются эти показатели в зависимости от признака, по которому было осуществлено разделение. Такое разделение мы можем назвать группировкой данных.

Возьмем информацию о числе единиц проданного товара в пяти первых сделках для стран Пакистан, Германия, Мексика.

In [ ]:
fs_pakistan = fs[fs["Country"]=="Pakistan"][["Country", "UnitsSold"]].head()
fs_germany = fs[fs["Country"]=="Germany"][["Country", "UnitsSold"]].head()
fs_mexico = fs[fs["Country"]=="Mexico"][["Country", "UnitsSold"]].head()

Найдем среднее значение *UnitsSold* в каждой группе

In [ ]:
print(fs_pakistan["UnitsSold"].mean())
print(fs_germany["UnitsSold"].mean())
print(fs_mexico["UnitsSold"].mean())

Объединим три этих таблицы в одну. Параметр `ignore_index=True` означает, что после объединения индекс надо перестроить.

In [ ]:
fs_joined = pd.concat([fs_pakistan, fs_germany, fs_mexico], axis=0, ignore_index=True)
fs_joined

Применим функцию *groupby()*

In [ ]:
fs_joined.groupby(by="Country")["UnitsSold"].mean()

## Функция *groupby()*

Ранее мы уже рассмотрели простейшую группировку с помощью функции *value_counts()*, которая возвращает серию, содержащую количество уникальных значений в группах. Кратко вспомним, как работает *value_counts()* на примере. Пусть имеются сведения об успеваимости студентов: имя студента, учебная группа, оценка в баллах в первом и втором семестрах.

In [ ]:
students=pd.DataFrame({'Name':
                    ['Иван','Петр','Мария','Василий','Анна','Василиса','Дарья','Николай','Вероника'],
                    'Group':['Ф1','Ф1','Ф2','Ф2','Ф1','Ф2','Ф2','Ф2','Ф1'],
                    'Mark_1':[19,18,17,21,24,18,22,19,18],
                    'Mark_2':[25,30,26,26,21,23,25,29,31]})

Распечатаем датафрейм с данными о студентах, предварительно добавив столбец - суммарную оценку

In [ ]:
students['Mark_sum'] = students['Mark_1'] + students['Mark_2']
students

Применим *value_counts()* к столбцам *Group* и *Mark_1*

In [ ]:
students['Group'].value_counts()

Результат: уникальными значениями столбца *Group* являются Ф1 и Ф2, которые встречаются 4 и 5 раз соответственно. Применим *value_counts()* к численному столбцу *Mark_1*.

In [ ]:
students['Mark_1'].value_counts()

Функция *value_counts()* сгруппировала студентов по группам и по оценкам. Предположим теперь, что нас интересует не количество студентов, а сумма баллов. Это можно сделать с помощью известного нам уже способа отбора по условию и применения статистической функции суммы. Для первого семестра получим:

In [ ]:
print(students[students['Group']=='Ф1']['Mark_1'].sum())
print(students[students['Group']=='Ф2']['Mark_1'].sum())

Для таких целей удобнее применять функцию *groupby()*

In [ ]:
students.groupby(['Group'])['Mark_1'].sum()

Рассмотрим более подробно работу функции *groupby()*. Выполним команду `students.groupby(['Group'])`

In [ ]:
students.groupby(['Group'])

Мы получили объект, который хранит сведения о том, какие строки относятся к каким значениям группируемого столбца. Чтобы распечатать эти сведения выполним эту же команду с атрибутом `.groups`

In [ ]:
students.groupby(['Group']).groups

Теперь мы узнали, что к группе Ф1 относятся строки [0, 1, 4, 8], а к Ф2 [2, 3, 5, 6, 7]. Эту информацию использует *pandas* для того, чтобы разбить данные на подгруппы и в каждой подгруппе рассчитать что-либо для нас.

Для того, чтобы провести суммирование (или выполнить другую агрегирующую функцию) по нескольким столбцам, их надо явно указать. Проведем суммирование баллов в каждой учебной группе отдельно в первом и во втором семестрах.

In [ ]:
students_groupped = students.groupby(['Group'])[['Mark_1','Mark_2']].sum()
students_groupped

Объект *studens_groupped* - это *DataFrame*.

In [ ]:
type(students_groupped)

Как известно, к элементам таблицы *pandas* можно получить доступ с помощью `.loc[]` и `iloc[]`. Например, чтобы получить сумму баллов студентов Ф2 во втором семестре введем команду

In [ ]:
students_groupped.loc['Ф2','Mark_2']

Если не уточнять, по какому столбцу проводится суммирование (или другая агрегирующая функция), то получим сумму по всем числовым столбцам

In [ ]:
students_groupped = students.groupby(['Group']).sum()
students_groupped

Студенты какой группы получили в сумме больше баллов? Для ответа отсортируем с помощью `.sort_values()`  таблицу сгруппированных значений по убыванию суммы баллов. Это группа Ф2.

In [ ]:
students_groupped.sort_values('Mark_sum',ascending=False)

## Другие агрегирующие функции

Еще раз распечатаем набор данных.

In [ ]:
fs.head()

Также распечатаем его описание.

In [ ]:
fs_description=pd.read_csv('../Data/FinancialSampleFields.csv', sep=';')
fs_description

### Функции *max(), min()*

Функция *max()* подсчитывает максимальное значение в серии. Найдем максимальную скидку с группировкой по стране.

In [ ]:
fs.groupby("Country")[["Discounts"]].max()

Функция *min()* подсчитывает минимальное значение в серии. Найдем минимальное число единиц  товара с группировкой по сегменту рынка и отсортируем по убыванию.

In [ ]:
fs.groupby('Segment')['UnitsSold'].min().sort_values(ascending=False)

### Функция *nunique()*

Функция *nunique()* позволяет посчитать количество уникальных значений в серии. Её лучше всего применять к столбцам, в которых хранятся категориальные данные.

In [ ]:
fs.groupby("Country")[["Segment", "Product", "DiscountBand"]].nunique()

### Функция *count()*

Позволяет посчитать количество непустых элементов в группе. Найдём число сделок в каждом сегменте. Число непустых значений будем подсчитывать по столбцу *Id*. Если конкретный столбец явно не указать, то *pandas* выведет на печать все столбцы таблицы.

In [ ]:
fs.groupby('Segment')['Id'].count()

Аналогичный результат получим с помощью *value_counts()*. Сравните:

In [ ]:
fs['Segment'].value_counts()

### Функция *median()*

Находит медианное значение. Рассчитаем среднее и медианное значение прибыли по сделкам каждого производителя. Для этого воспользуемся функцией *.agg()*

In [ ]:
fs.groupby('Product')['Profit'].agg(['mean','median'])

### Функция *std()*
Рассчитывает стандартное отклонение. Найдем среднее, медианное значения и стандартное отклонение себестоимости и прибыли от продажи товара в каждом сегменте.

In [ ]:
fs.groupby('Segment')[['COGS', 'Profit']].agg(['mean','median','std'])

## Сводные таблицы

Что такое "сводная таблица" и  чем она отличается от "плоской" таблицы? Как правило, значениями плоской таблицы являются атрибуты параметров объектов, расположенных в строках. Названия параметров расположены в столбцах. Например, каждая строка *Financial Sample* является объектом - сделкой по продаже товара. Параметры сделки указаны в столбцах. на пересечении строки и столбца содержится значение параметра для конкретного объекта. 

Значениями сводной таблицы является результат вычисления агрегирующей функции - сгруппированные данные. При этом показатели, на основе которых происходит объединение данных, расположены в заголовках строк и столбцов.

### Функция *pivot_table()*

Функция *pivot_table()* позволяет быстро и просто составлять сводные таблицы
Согласно документации https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html 

```
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
```

параметрами функции являются:

- data - исходная таблица с данными
- index - строки
- columns - столбцы
- values - агрегируемый столбец исходной таблицы
- aggfunc - функция агрегации
- margins - вывод суммирующих строки и столбца (логическое значение)
- fill_value - значение, на которое следует заменять пустые значения
- dropna - удалять или нет результаты, в которых все значения пустые (логическое)

Составим сводную таблицу, в которой отобразим макимальную скидку, предоставленную каждым производителем в каждом сегменте рынка. Расположим названия сегментов по строкам, а производителей по столбцам. В качестве агрегируемого столбца укажем *Discounts* -  столбец, в котором содержится значение скидки. Функция агрегации "max".

In [ ]:
pd.pivot_table(fs, values='Discounts', index='Segment', columns=['Product'], aggfunc='max')

Добавим для производителей уровень группирвки по стране. Для экономии места и большей наглядности ограничимся двумя странами - Германией и Францией.

In [ ]:
fs_GF = fs[fs['Country'].isin(['Germany','France'])]

In [ ]:
pd.pivot_table(fs_GF, values='Discounts', 
               index='Segment', columns=['Country','Product'], aggfunc='max')

### Доступ к элементам сводной таблицы

Как получить доступ к отдельному столбцу или столбцам? Извлечем из сводной таблицы значение максимальной скидки, предоставленной производителем *Amarilla* для правительственного сегмента Франции. Это число находится в строке *Government* и столбце *France Amarilla*.

Присвоим результату вычисления сводной таблицы  имя *pvt* и выполним команду `type(pvt)` чтобы узнать, какого типа этот объект.

In [ ]:
pvt=pd.pivot_table(fs_GF, values='Discounts', 
               index='Segment', columns=['Country','Product'], aggfunc='max')

In [ ]:
type(pvt)

*pvt* - таблица *pandas*, значит у нее есть атрибуты *columns* и *index*.

In [ ]:
pvt.columns

In [ ]:
pvt.index

Тогда применим `.loc[]` для доступа к искомому значению скидки 

In [ ]:
pvt.loc['Government', ( 'France',  'Amarilla')]

## Консолидация: объединение данных с помощью сводной таблицы

Задача.
Имеются csv-файлы с информацией об изменении цены акций. 



1.   [GAZA](https://drive.google.com/file/d/1bw3d01WXw5_N1skmnXwZUHFeT_xTiU1c/view?usp=sharing)
2.   [KMAZ](https://drive.google.com/file/d/1JKendyxL4tF4-JrA7DzkHKzgSYjHtVkT/view?usp=sharing)
3.   [ROSN](https://drive.google.com/file/d/18ImO_nT9yaghfkLfe3UNkmqfwszZI1Si/view?usp=sharing)

*Источник: [finam.ru](https://www.finam.ru/)*

Задача. Объединить эти данные в одну таблицу, в строках разместить дату, в столбцах код компании, в ячейках цену акции в соответствующий день.

In [ ]:
GAZA=pd.read_csv('../Data/t_GAZA_200401_201130.csv',sep=';')
KMAZ=pd.read_csv('../Data/t_KMAZ_200401_201130.csv',sep=';')
ROSN=pd.read_csv('../Data/t_ROSN_200401_201130.csv',sep=';')

Ознакомимся кратко с этими источниками, отобразив первые пять строк.

In [ ]:
GAZA.head()

In [ ]:
KMAZ.head()

In [ ]:
ROSN.head()

Объединение таблиц с помощью <TT>.concat()</tt>

In [ ]:
# "Объединение" таблиц проводится с помощью .concat()
join=pd.concat([GAZA,KMAZ,ROSN], axis=0)

# в объединенной таблице преобразуем дату к типу даты
join['<DATE>']=pd.to_datetime(join['<DATE>'],dayfirst=True) 

# и перестроим индекс
join.index=range(len(join)) 
join

Проверка по числу строк и столбцов.

In [ ]:
# Проверка. В объединенной таблице столько же столбцов, сколько в любой из исходных таблиц 
# и столько строк, сколько в сумме у всех объединяесыъ таблиц.
print(GAZA.shape)
print(KMAZ.shape)
print(ROSN.shape)
print(join.shape)

Краткая информация об объединенной таблице

In [ ]:
join.info()

Создадим сводную таблицу

In [ ]:
pvt=join.pivot_table(index='<DATE>', columns='<TICKER>', values='<CLOSE>')
pvt

Проведена консолидация данных: объединенная таблица содержит цену акций из всех трех источников и только в одном столбце момент времени.

Проверка. Подсчитаем число строк в консолидированной таблице. Оно совпадает с числом строк в любой из исходных таблиц.

In [ ]:
len(pvt)

## Выполните контрольные задания

### Исходный набор данных

Загрузите в pandas.DataFrame набор данных *ListingAm.csv* и файл *ListingAmDescription.csv* с описанием полей этого набора данных.

In [ ]:
# Ваш код здесь


### Выполните задания
1. Найдите среднее число коек в каждом районе Амстердама.

In [ ]:
# Ваш код здесь


2. Найдите среднюю оценку чистоты (качества уборки) помещения для каждого типа собственности.

In [ ]:
# Ваш код здесь


3. Найдите среднюю цену размещения в для каждого типа комнаты в районе *Westerpark*.

In [ ]:
# Ваш код здесь


4. С помощью *groupby()* найдите количество объектов размещения для каждого типа комнаты в *Noord-West*.

In [ ]:
# Ваш код здесь


5. Выполните задание 4 с помощью *.value_counts()*

In [ ]:
# Ваш код здесь


6. Найдите среднее и медианное значения числа спален и коек в разрезе по типу собственности в *Bos en Lommer*.

In [ ]:
# Ваш код здесь


7. Используя результат предыдущего задания, определите:

    а) для каких типов собственности средние и медианные значения числа коек равны?<br>
    б) для каких типов собственности медианное значение числа спален превышает среднее значение?

In [ ]:
# Ваш код здесь


8. С помощью сводной таблицы найдите минимальную цену размещения каждого типа комнаты а каждом районе.

In [ ]:
# Ваш код здесь


9. Какие владельцы владеют пятью и более объектами недвижимости? Распечатайте их идентификаторы  вместе с именами и числом объектов в порядке убывания числа объектов.

In [ ]:
# Ваш код здесь


## Решение

In [51]:
am = pd.read_csv('../Data/ListingsAm.csv', sep=';')
am_desc = pd.read_csv('../Data/ListingsAmDescription.csv', sep=',')

In [54]:
am

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2010-03-02,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58
1,27886,https://www.airbnb.com/rooms/27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,t,Centrum-West,Private room in houseboat,Private room,1.5 baths,...,2012-01-09,2020-07-25,99,10.0,10.0,10.0,10.0,10.0,10.0,2.01
2,28871,https://www.airbnb.com/rooms/28871,Comfortable double room,124245,Edwin,t,Centrum-Oost,Private room in apartment,Private room,1 shared bath,...,2010-08-22,2020-09-20,97,10.0,10.0,10.0,10.0,10.0,10.0,2.68
3,29051,https://www.airbnb.com/rooms/29051,Comfortable single room,124245,Edwin,t,Centrum-Oost,Private room in apartment,Private room,1 shared bath,...,2011-03-16,2020-08-28,95,10.0,10.0,10.0,10.0,10.0,9.0,4.05
4,31080,https://www.airbnb.com/rooms/31080,2-story apartment + rooftop terrace,133488,Nienke,f,Zuid,Entire apartment,Entire home/apt,1 bath,...,2011-08-06,2017-10-16,95,9.0,10.0,10.0,10.0,9.0,9.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,45168887,https://www.airbnb.com/rooms/45168887,The Weber Collection - Luxurieus Design Studio...,178187873,Marcel And Pauline,f,Centrum-Oost,Entire serviced apartment,Entire home/apt,1 bath,...,2020-09-11,2020-11-01,76,7.0,7.0,7.0,8.0,9.0,8.0,1.61
3531,45169008,https://www.airbnb.com/rooms/45169008,The Weber Collection - Luxurieus Design Studio...,178187873,Marcel And Pauline,f,Centrum-Oost,Entire serviced apartment,Entire home/apt,1 bath,...,2020-09-02,2020-09-28,87,10.0,9.0,10.0,8.0,10.0,10.0,0.88
3532,45836338,https://www.airbnb.com/rooms/45836338,The Weber Collection - Luxurieus Design Studio...,178187873,Marcel And Pauline,f,Centrum-Oost,Entire apartment,Entire home/apt,1 bath,...,2020-10-18,2020-11-17,70,7.0,8.0,7.0,8.0,9.0,8.0,3.21
3533,45854688,https://www.airbnb.com/rooms/45854688,The Weber Collection - Luxurieus Design Studio...,178187873,Marcel And Pauline,f,Centrum-Oost,Entire serviced apartment,Entire home/apt,1 bath,...,2020-10-24,2020-11-04,100,10.0,10.0,10.0,9.0,10.0,10.0,1.80


In [53]:
am_desc

,Атрибут,Значение
0,id,идентификатор размещения
1,listing_url,адрес веб-страницы
2,name,название размещения
3,host_id,идентификатор владельца
4,host_name,имя владельца
5,host_is_superhost,супервладелец
6,neighbourhood_cleansed,местоположение
7,property_type,тип собственности
8,room_type,тип комнаты
9,bathrooms_text,текстовое описание ванной


In [55]:
#1

am.groupby('neighbourhood_cleansed')['beds'].mean()

neighbourhood_cleansed
Bijlmer-Centrum                           2.000000
Bos en Lommer                             1.497487
Buitenveldert - Zuidas                    2.108108
Centrum-Oost                              1.752874
Centrum-West                              1.814583
De Aker - Nieuw Sloten                    1.888889
De Baarsjes - Oud-West                    1.627512
De Pijp - Rivierenbuurt                   1.686275
Geuzenveld - Slotermeer                   1.750000
IJburg - Zeeburgereiland                  2.975000
Noord-Oost                                2.230769
Noord-West                                2.106667
Oostelijk Havengebied - Indische Buurt    1.767123
Osdorp                                    1.526316
Oud-Noord                                 2.747967
Oud-Oost                                  1.560748
Slotervaart                               1.712329
Watergraafsmeer                           2.142857
Westerpark                                1.555985
Zuid    

In [56]:
#2

am.groupby('property_type')['review_scores_cleanliness'].mean()

property_type
Boat                                   9.611111
Entire apartment                       9.511542
Entire bed and breakfast              10.000000
Entire condominium                     9.553191
Entire cottage                        10.000000
Entire guest suite                     9.571429
Entire guesthouse                     10.000000
Entire house                           9.465241
Entire loft                            9.476923
Entire place                           9.500000
Entire serviced apartment              9.285714
Entire townhouse                       9.561905
Entire villa                           9.500000
Houseboat                              9.543478
Private room                          10.000000
Private room in apartment              9.491841
Private room in bed and breakfast      9.474227
Private room in boat                   9.666667
Private room in bungalow              10.000000
Private room in cabin                  9.000000
Private room in condominiu

In [57]:
#3

am[am['neighbourhood_cleansed']=='Westerpark'].groupby('room_type')['price'].mean()

room_type
Entire home/apt    155.139013
Hotel room         157.000000
Private room        84.666667
Name: price, dtype: float64

In [58]:
#4

am[am['neighbourhood_cleansed']=='Noord-West'].groupby('room_type')['id'].count()

room_type
Entire home/apt    45
Hotel room          1
Private room       28
Shared room         1
Name: id, dtype: int64

In [59]:
#5

am[am['neighbourhood_cleansed']=='Noord-West']['room_type'].value_counts()

Entire home/apt    45
Private room       28
Shared room         1
Hotel room          1
Name: room_type, dtype: int64

In [60]:
#6

BB = am[am['neighbourhood_cleansed']=='Bos en Lommer'].groupby('property_type')[['bedrooms','beds']].agg(['mean','median'])
BB

bedrooms             beds       
                                       mean median      mean median
property_type                                                      
Entire apartment                   1.391026    1.0  1.484472    1.0
Entire condominium                 1.000000    1.0  1.000000    1.0
Entire house                       1.800000    2.0  2.200000    2.0
Entire loft                        2.000000    2.0  2.000000    2.0
Entire townhouse                   3.000000    3.0  3.000000    3.0
Houseboat                          1.000000    1.0  2.000000    2.0
Private room in apartment          1.076923    1.0  1.269231    1.0
Private room in bed and breakfast  1.000000    1.0  3.000000    3.0
Private room in guest suite        1.000000    1.0  1.000000    1.0

In [61]:
#7а

BB[BB[('beds', 'mean')]==BB[('beds', 'median')]]

bedrooms        beds       
                                      mean median mean median
property_type                                                
Entire condominium                     1.0    1.0  1.0    1.0
Entire loft                            2.0    2.0  2.0    2.0
Entire townhouse                       3.0    3.0  3.0    3.0
Houseboat                              1.0    1.0  2.0    2.0
Private room in bed and breakfast      1.0    1.0  3.0    3.0
Private room in guest suite            1.0    1.0  1.0    1.0

In [62]:
#7б

BB[BB[('bedrooms', 'median')]>BB[('bedrooms', 'mean')]]

bedrooms        beds       
                  mean median mean median
property_type                            
Entire house       1.8    2.0  2.2    2.0

In [63]:
#8

pvt=pd.pivot_table(am, values='price', 
                   index='neighbourhood_cleansed', columns='room_type', aggfunc='min')
pvt

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_cleansed,,,,
Bijlmer-Centrum,78.0,NaN,NaN,NaN
Bos en Lommer,37.0,NaN,28.0,NaN
Buitenveldert - Zuidas,74.0,NaN,45.0,NaN
Centrum-Oost,56.0,96.0,22.0,NaN
Centrum-West,51.0,46.0,38.0,35.0
De Aker - Nieuw Sloten,65.0,NaN,29.0,NaN
De Baarsjes - Oud-West,40.0,41.0,19.0,51.0
De Pijp - Rivierenbuurt,35.0,55.0,26.0,45.0
Geuzenveld - Slotermeer,79.0,NaN,25.0,60.0


In [64]:
#9

H = am.groupby(['host_id', 'host_name'])['host_id'].count().sort_values(ascending=False)
H[H>4]

host_id    host_name                     
65859990   Martijn                           34
1464510    Wendy And Bram                    28
7594884    Ruben, Sarah & Lindy              15
113977564  Willem                            10
178187873  Marcel And Pauline                 9
13405827   Cap Homes                          9
203731852  SWEETS Hotel                       8
21167882   Pauline                            8
3899684    City Apartments Amsterdam          7
67005410   Feliciano                          7
5833627    Greg                               7
10076897   Geraldine                          6
273266448  Vlad                               6
5796250    Remco                              6
11969034   Raul                               6
16419683   Laurent                            5
198405490  Suzanne                            5
8558897    Alex                               5
237150996  St Christophers At The Winston     5
300888539  Cosimo                             